<a href="https://colab.research.google.com/github/amankiitg/Agents/blob/main/Final_CrewAI_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Outreach Campaign

We learn how to create custom tools for a customer outreach campaign. A Good tool has 3 key elements:

- Versatility
- Caching
- Fault Tolerance

Note: CrewAI only works with Python version >=3.10 and <=3.13

In [1]:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


## Step 01. Installing CrewAI packages

In [2]:
%%capture
!pip install crewai

In [3]:
%%capture
!pip install 'crewai[tools]'

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing CrewAI library (Agent, Task and Crew module)

In [5]:
from crewai import Agent, Task, Crew

To use Agents, we need to set env. variables (key, model name)

In [6]:
import os
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('openaikey')
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ['SERPER_API_KEY'] = userdata.get('serperkey')

## Step 03. Creating Agents

In [7]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [8]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Step 04. Creating Tools

### CrewAI Tools

- **SerperDevTool** : Used for Google Search
- **DirectoryReadTool** : Open a directory
- **FileReadTool** : Read a file

In [9]:
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [10]:
# local directory 'instructions' contains markdown files
# markdown files (.md) having instructions on how to deal with company of different sizes

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
serper_dev_tool = SerperDevTool()

### Custom Tool (SentimentAnalysisTool)

- Create a custom tool using crewAI's BaseTool class

In [11]:
from crewai.tools import BaseTool


- Every Tool needs to have a `name` and a `description`.
- Tool logic written in `_run` function
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [12]:
class SentimentAnalysisTool(BaseTool):
    name : str = "Sentiment Analysis Tool"
    description : str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, argument : str) -> str:
        return "positive"


In [13]:
sentiment_analysis_tool = SentimentAnalysisTool()


## Step 05. Creating Tasks

- Lead Profiling Task usign crewAI Tools

In [14]:
lead_profiling = Task (
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[serper_dev_tool, directory_read_tool, file_read_tool],
    agent=sales_rep_agent
)

- The personalized outreach task is created using custom tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search tool)

In [15]:
personalized_outreach_task = Task (
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"

    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[serper_dev_tool, sentiment_analysis_tool],
    agent=lead_sales_rep_agent
)

## Step 06.  Creating a crew

In [16]:
crew = Crew(
    tasks=[lead_profiling, personalized_outreach_task],
    agents=[sales_rep_agent, lead_sales_rep_agent],
    memory=True,
    verbose=False
)

/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


## Kicking off the crew

In [17]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Thought: To obtain a comprehensive report on DeepLearningAI, I need to start by finding general       │
│  information about the company, identify key personnel, learn about recent milestones, and understand their     │
│  specific needs that align with our solutions. The best tool to initiate this search and gather data is         │
│  Serper.                                                                                                        │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"DeepLearningAI company profile\"}"                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'DeepLearningAI company profile', 'type': 'search', 'num': 10, 'engine':            │
│  'google'}, 'organic': [{'title': 'About - DeepLearning.AI', 'link': 'https://www.deeplearning.ai/about/',      │
│  'snippet': 'DeepLearning.AI is an education technology company that is empowering the global workforce to      │
│  build an AI-powered future through world-class education, hands-on training, and a collaborative community     │
│  .', 'position': 1}, {'title': 'DeepLearning.AI - LinkedIn', 'link':                                            │
│  'https://www.linkedin.com/company/deeplearningai', 'snippet': 'DeepLearning.AI is making a world-class AI      │
│  education accessible to people around the globe. DeepLearning.AI was founded by Andrew Ng, a global leader in  │
│  AI.', 'position': 2}, {'title': 'Deeplearning.ai - Crunchbase Company Profile & Funding', 'link':              │
│  'https://www.crunchbase.com/organization/deeplearning-ai', 'snippet': 'Deeplearning.ai is company that         │
│  explores the frontier of AI. The company is founded by Andrew Ng, the former Chief Scientist of Baidu.',       │
│  'position': 3}, {'title': 'Deeplearning - 2025 Company Profile, Funding & Competitors', 'link':                │
│  'https://tracxn.com/d/companies/deeplearning/__SekrcD6K_o4Cvm4SgstSYBIp1zHk-3FzPsR3SkeAgnE', 'snippet':        │
│  'Deeplearning is a funded company based in Palo Alto (United States), founded in 2017 by Andrew Ng. It         │
│  operates as an Online platform providing multi- ...', 'position': 4}, {'title': 'deeplearning.ai Information   │
│  - RocketReach', 'link': 'https://rocketreach.co/deeplearningai-profile_b4556bbefc954d11', 'snippet':           │
│  'deeplearning.ai is an Artificial Intelligence, Data and Analytics, and Deep Learning company located in Palo  │
│  Alto, California with 216 employees.', 'position': 5}, {'title': 'DeepLearning.AI: Start or Advance Your       │
│  Career in AI', 'link': 'https://www.deeplearning.ai/', 'snippet': 'Join over 7 million people learning how to  │
│  use and build AI. AI Courses and Specializations. Build a foundation of machine learning and AI skills.',      │
│  'position': 6, 'sitelinks': [{'title': 'Abou...                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Thought: The initial search provided an excellent overview of DeepLearningAI, including key details   │
│  about their founding and operations. Next, I should gather more focused information on their key personnel,    │
│  latest milestones, and any specific needs they might have which our solutions can address.                     │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Thought: I need to tailor the campaign's narrative to reflect a deep knowledge of DeepLearningAI's culture,    │
│  values, and current goals. Since Andrew Ng is a key figure, personalization should highlight his leadership    │
│  and vision, connecting these elements with CrewAI's technology. Given the necessity of tone congruence and     │
│  engaging content, I should first ensure that any drafted message maintains a positive sentiment.               │
│                                                                                                                 │
│  Using Tool: Sentiment Analysis Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"argument\": \"Dear Dr. Andrew Ng, We at CrewAI are admirers of DeepLearningAI's vision to democratize adv  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  positive                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here are a series of personalized email drafts targeting Andrew Ng that align with DeepLearningAI's            │
│  achievements and future goals:                                                                                 │
│                                                                                                                 │
│  **Email 1: Introduction and Congratulations**                                                                  │
│  Subject: Celebrating Your Achievements and Proposing a Future Together                                         │
│                                                                                                                 │
│  Dear Dr. Ng,                                                                                                   │
│                                                                                                                 │
│  I hope this message finds you thriving. I am [Your Name], the Lead Sales Representative at CrewAI. First and   │
│  foremost, congratulations on surpassing several million learners and expanding DeepLearningAI's educational    │
│  reach! Your recent initiatives in courses like Generative AI and their applications in climate science are     │
│  impressive and truly pioneering.                                                                               │
│                                                                                                                 │
│  At CrewAI, we specialize in enhancing educational platforms through data-driven insights and AI technology,    │
│  aiming to make learning more interactive and impactful. Our tools could potentially elevate the learner        │
│  experience in your new courses, ensuring that education is not only accessible but also deeply engaging.       │
│                                                                                                                 │
│  Would you be available for a brief call next week to discuss potential synergies? We are excited about the     │
│  possibility of contributing to your vision and enriching the global AI learning landscape together.            │
│                                                                                                                 │
│  Warm regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│  **Email 2: Detailing Solutions and Benefits**                                                                  │
│  Subject: Tailored Solutions to Amplify DeepLearningAI's Impact                                                 │
│                                                                                                                 │
│  Dear Dr. Ng,                                                                                                   │
│                                                                                                                 │
│  Following our last interaction, I've delved deeper int

/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [18]:
from IPython.display import Markdown
import re

content = result.raw
markdown_content = re.sub(r'```(?:markdown)?\n?(.*?)\n?```', r'\1', content, flags=re.DOTALL).strip()
Markdown(markdown_content)

Here are a series of personalized email drafts targeting Andrew Ng that align with DeepLearningAI's achievements and future goals:

**Email 1: Introduction and Congratulations**
Subject: Celebrating Your Achievements and Proposing a Future Together

Dear Dr. Ng,

I hope this message finds you thriving. I am [Your Name], the Lead Sales Representative at CrewAI. First and foremost, congratulations on surpassing several million learners and expanding DeepLearningAI's educational reach! Your recent initiatives in courses like Generative AI and their applications in climate science are impressive and truly pioneering.

At CrewAI, we specialize in enhancing educational platforms through data-driven insights and AI technology, aiming to make learning more interactive and impactful. Our tools could potentially elevate the learner experience in your new courses, ensuring that education is not only accessible but also deeply engaging.

Would you be available for a brief call next week to discuss potential synergies? We are excited about the possibility of contributing to your vision and enriching the global AI learning landscape together.

Warm regards,

[Your Name]

**Email 2: Detailing Solutions and Benefits**
Subject: Tailored Solutions to Amplify DeepLearningAI's Impact

Dear Dr. Ng,

Following our last interaction, I've delved deeper into specific challenges in AI education and how CrewAI can address these effectively. Our AI-driven engagement tools are designed to optimize learner interaction through custom learning paths, while our robust analytics solutions can dissect educational impact on a granular level.

By integrating our technology with DeepLearningAI's platform, we can offer:
- Enhanced personalization and engagement in learning modules.
- Advanced analytics for precise measurement of course effectiveness and learner satisfaction.
- Scalable solutions to meet the needs of your growing global audience.

We believe these enhancements will not only support your educational goals but also help set new benchmarks in the AI learning industry.

Looking forward to your thoughts,

[Your Name]

**Email 3: Proposal for a Pilot Program**
Subject: Proposal for a Pilot Program: Transforming AI Education Together

Dear Dr. Ng,

Building on our discussions, CrewAI proposes to initiate a pilot program with DeepLearningAI. This program would focus on integrating our AI-driven tools in a select number of your courses to demonstrate real-world improvements in learning outcomes.

Our team is prepared to work closely with yours to ensure a seamless integration and detailed evaluation of results. We anticipate that the successful implementation of this pilot will pave the way for a broader partnership between our organizations, driving forward our shared vision for global AI education.

Please let us know a suitable time to further discuss this exciting opportunity.

Eagerly awaiting your partnership, 

[Your Name]

These drafts are designed to engage, inform, and propose actionable steps towards a collaboration that aligns directly with the strategic directions of both DeepLearningAI and CrewAI.